#SASA Prediction Code

This code trains a multi-layer perceptron regression to predict SASA values based on chemical shift data.

## Setup the code with imports / function definitions

In [0]:
# All import statements
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from scipy.stats import randint as sp_randint
from scipy.stats import expon as sp_expon
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
import sklearn

from sklearn.linear_model import MultiTaskLasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor

import re
import numpy as np
import pandas as pd
import io
import requests
import warnings

First, we define some necessary functions for formatting the dataset.

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def one_hot(cs):
  '''
  This function encodes the resnames so that there are now 4 columns 
  '''
  one_hot = pd.get_dummies(cs['resname'])
  cs = cs.join(one_hot)
  return(cs)

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'ADE', 'CYT', 'GUA', 'URA', 'sasa-All-atoms', 'sasa-Total-Side',
       'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'sasa-All-atoms', 'sasa-Total-Side',
       'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar', 'ADE', 'CYT', 'GUA', 'URA']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'sasa-All-atoms', 'sasa-Total-Side',
       'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar', 'resid', 'ADE', 'CYT', 'GUA', 'URA']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, dtype='object', columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features, dtype='object')
  info = pd.DataFrame(cs[retain].values, dtype='object', columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def create_training_testing(cs, leave_out = "2KOC", target_name = ['sasa-All-atoms', 'sasa-Total-Side',
       'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar'], neighbors = 2, drop_names = ['id', 'sasa-All-atoms', 'sasa-Total-Side',
       'sasa-Main-Chain', 'sasa-Non-polar', 'sasa-All-polar', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the stacking and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
    
  '''  
  # Start: your code
  
  cs_new = pd.DataFrame()
  
  for id in c.id.unique():
    cs_id = get_cs_all(cs,id)
    cs_new = pd.concat([cs_new,get_cs_features_rna(cs_id, neighbors)], axis = 0)
  
  
  # End: your code
  return(cs_new)

## Prepare the datasets

Download and prepare chemical shift data

In [24]:
warnings.filterwarnings("ignore")   

# load initial data
url="https://drive.google.com/uc?id=1e-SHtWDtg4mD_th3_4Jmq9r1iiQC32wT"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep=' ')
print("[INFO]: loaded data")

# Drop extraneous columns that are unneeded for prediction
c = c.drop(['Unnamed: 0','base_pairing', 'orientation', 'sugar_puckering', 'pseudoknot', 'stacking'], axis = 1)

# One-hot encode the resname data so it can be used in the model
c_new = one_hot(c)

[INFO]: loaded data


Download and prepare SASA datasest

In [25]:
warnings.filterwarnings("ignore")   

# load initial data
url="https://drive.google.com/uc?export=download&id=1hn5uEIcP2sAwyTLeoGXNMdLm61T_Ew7c"
s=requests.get(url).content
c_sasa=pd.read_csv(io.StringIO(s.decode('utf-8')), sep=' ')
print("[INFO]: loaded data")

# The SASA file contains 5 extra rows, so they must be deleted
c_sasa = c_sasa.drop(c_sasa.index[[235, 1254, 1280, 1947, 2436, 2476]])
c_sasa = c_sasa.reset_index(drop=True)

# remove redundant columns
c_sasa = c_sasa.drop(['id', 'resid','resname'], axis = 1)

[INFO]: loaded data


Combine chemical shifts and sasa datasets together

In [0]:
c_concat = pd.concat([c_new, c_sasa], axis=1)

Prepare the new dataset to contain neighbor chemical shifts as well

In [0]:
# The ideal number of neighbors found in prior assignment was Neighbors = 4
NEIGHBORS = 4
cs_all = get_cs_features_rna_all(c_concat, neighbors = NEIGHBORS)

## Determine best hyperparameters

We need to identify some good hyperparameters to use. Below, we set up a distribution space that will be used to search through.

In [0]:
# Parameter space distribution allows for random range of integers
min_size, max_size = 10, 100
parameter_space_distribution = {
    'hidden_layer_sizes': [(sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),sp_randint.rvs(min_size, max_size)), (sp_randint.rvs(min_size, max_size),)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': sp_expon(scale=.01),
    'learning_rate': ['constant','adaptive'],
    'learning_rate_init': sp_expon(scale=.01),
}

Now, we set up the training and testing datasets so that we can use `RandomizedSearchCV` to search the parameter space for a suitable set of hyperparameters.

Note: Sometimes the random search results in an error due to some value being `NaN`. We adjusted the parameter space distribution until we were able to search through 20 folds without any error occuring. When running again, we can't guarantee that the error won't occur at some point. After a few tries, you should be able to eventually get a set of best parameters without any error occuring.


In [61]:
# Seperate data into training and testing set
trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = '1A60', neighbors = NEIGHBORS)
print("[INFO]: created training and testing data structures")

# setup scaler and transform the X data
scaler = StandardScaler()
scaler.fit(trainX)
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)
print("[INFO]: scaled the features")

# set the model to be a multilayer perceptron regressor
model = MLPRegressor()

# Random search for best hyperparameter
n_iter_search = 10
random_search = RandomizedSearchCV(model, param_distributions=parameter_space_distribution, n_iter=n_iter_search, cv=2, verbose = 1)
random_search.fit(trainX_scaled, trainy)
random_search.best_params_

[INFO]: created training and testing data structures
[INFO]: scaled the features
Fitting 2 folds for each of 10 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.6min finished


{'activation': 'tanh',
 'alpha': 0.005654169847189843,
 'hidden_layer_sizes': (181,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.006401053696231486,
 'solver': 'adam'}

Good parameters found previously were:
```
activation = 'tanh',
alpha = 0.005654169847189843,
hidden_layer_sizes = (181,),
learning_rate = 'constant',
learning_rate_init = 0.006401053696231486,
solver = 'adam'
```

## Train the data and calculate mean R2 score

The next section of code is where the fit and evaluation occur. For each loop, one of the IDs are left out to use as a testing set. This will loop through every ID and store the r2 scores of each loop into an `r2scores` array. At the end, it calculates the mean r2 scores.

In order to ensure that the same scores are reached every time the code is run, we don't use `random_search` explicitly here. Instead, we use a set of hyperparameters that were found previously from a `RandomizedSearchCV` run.

In [94]:
mlp_R2 = []
mtl_R2 = []
knr_R2 = []
gpr_R2 = []
dtr_R2 = []
combined_R2 = []
comb_red_R2 = []

id_array = cs_all.id.unique()
for id in id_array:
  trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = NEIGHBORS)
  print("[INFO]: created training and testing data structures for " + id)

  # setup scaler
  scaler.fit(trainX)
  trainX_scaled = scaler.transform(trainX)
  testX_scaled = scaler.transform(testX)
  print("[INFO]: scaled the features")

  # Fit tthe multilayer perceptron model
  mlp_model = sklearn.neural_network.MLPRegressor(activation = 'tanh',
                                                  alpha = 0.005654169847189843,
                                                  hidden_layer_sizes = (181,),
                                                  learning_rate = 'constant',
                                                  learning_rate_init = 0.006401053696231486,
                                                  solver = 'adam')
  mlp_model.fit(trainX_scaled,trainy)

  # fit the multitask lasso model
  mtl_model = MultiTaskLasso()
  mtl_model.fit(trainX_scaled,trainy)

  # fit the K Neighbors model
  knr_model = KNeighborsRegressor()
  knr_model.fit(trainX_scaled,trainy)

  # fit the gaussian process model
  gpr_model = GaussianProcessRegressor()
  gpr_model.fit(trainX_scaled,trainy)

  # fit the decision tree model
  dtr_model = DecisionTreeRegressor()
  dtr_model.fit(trainX_scaled,trainy)

  # Predict y values
  y_true = testy 
  mlp_ypred = mlp_model.predict(testX_scaled)
  mtl_ypred = mtl_model.predict(testX_scaled)
  knr_ypred = knr_model.predict(testX_scaled)
  gpr_ypred = gpr_model.predict(testX_scaled)
  dtr_ypred = dtr_model.predict(testX_scaled)

  # Combined model averages over baseline model y-value predictions
  combined_ypred = np.mean([mlp_ypred, mtl_ypred, knr_ypred, gpr_ypred, dtr_ypred],axis=0)
  comb_red_ypred = np.mean([mlp_ypred, mtl_ypred, knr_ypred],axis=0)

  # Record R2 scores for each of the models
  mlp_R2.append(sklearn.metrics.r2_score(y_true,mlp_ypred))
  mtl_R2.append(sklearn.metrics.r2_score(y_true,mtl_ypred))
  knr_R2.append(sklearn.metrics.r2_score(y_true,knr_ypred))
  gpr_R2.append(sklearn.metrics.r2_score(y_true,gpr_ypred))
  dtr_R2.append(sklearn.metrics.r2_score(y_true,dtr_ypred))
  combined_R2.append(sklearn.metrics.r2_score(y_true,combined_ypred))
  comb_red_R2.append(sklearn.metrics.r2_score(y_true,comb_red_ypred))

[INFO]: created training and testing data structures for 1A60
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1HWQ
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1JO7
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1KKA
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1L1W
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1LC6
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1LDZ
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1MFY
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1NA2
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1NC0
[INFO]: scaled the features
[INFO]: created training and testing data structures for 1OW9
[INFO]: scaled the features
[INFO]: cr

In [95]:
print('The mean scores are: ')
print('mlp:')
print(np.mean(mlp_R2))
print('mtl:')
print(np.mean(mtl_R2))
print('knr:')
print(np.mean(knr_R2))
print('gpr:')
print(np.mean(gpr_R2))
print('dtr:')
print(np.mean(dtr_R2))
print('combined:')
print(np.mean(combined_R2))
print('ccombined_r:')
print(np.mean(comb_red_R2))

The mean scores are: 
mlp:
0.3288263286289028
mtl:
0.36400947047729004
knr:
0.3624662505776763
gpr:
-22.572444930234152
dtr:
-0.3154695174488031
combined:
-0.547006942659105
ccombined_r:
0.44184868311959835


## Conclusion:

It is clear that the Gaussian Process Regressor and the Decision Tree Regressor do not work at all since their R2 scores are negative. The Combined Model ends up with a negative score as well because of this. The Reduced Combined Model is much more promising. Its value is consistently higher than any of the three models which it is comprised of. While the value is still low, it shows promise for future consideration.

Trial 1:

mlp:
0.3317651274871274

mtl:
0.36400947047729004

knr:
0.3624662505776763

gpr:
-22.572444930234152

dtr:
-0.25904090630243554

combined:
-0.5436059594629794

ccombined_r:
0.443197379356901

Trial 2:

mlp:
0.3422078372361123

mtl:
0.36400947047729004

knr:
0.3624662505776763

gpr:
-22.572444930234152

dtr:
-0.28796273804092515

combined:
-0.5379015072399548

ccombined_r:
0.4462537479044602

Trial 3:

mlp:
0.3288263286289028

mtl:
0.36400947047729004

knr:
0.3624662505776763

gpr:
-22.572444930234152

dtr:
-0.3154695174488031

combined:
-0.547006942659105

ccombined_r:
0.44184868311959835